<a href="https://www.kaggle.com/code/workhfailflearna/youtube-daily-trends?scriptVersionId=278451312" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
spark = SparkSession.builder.appName("Read CSV Example").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/15 20:31:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
csv_file_path = "/kaggle/input/youtube-new/CAvideos.csv"
schema = StructType([
		StructField("video_id", StringType() ,True),
		StructField("trending_date", StringType() ,True),
		StructField("title", StringType() ,True),
		StructField("channel_title", StringType() ,True),
		StructField("category_id", IntegerType() ,False),
		StructField("publish_time", StringType() ,True),
		StructField("tags", StringType() ,True),
		StructField("views", IntegerType() ,True),
		StructField("likes", IntegerType() ,True),
		StructField("dislikes", IntegerType() ,True),
		StructField("comment_count", IntegerType() ,True),
		StructField("thumbnail_link", StringType() ,True),
		StructField("comments_disabled", StringType() ,True),
		StructField("ratings_disabled", StringType() ,True),
		StructField("video_error_or_removed", StringType() ,True),
		StructField("description", StringType() ,True)
])
df = spark.read.schema(schema).csv(csv_file_path, header=True)
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [9]:
json_file_path = "/kaggle/input/youtube-new/CA_category_id.json"
categories = spark.read.option("multiLine", "true").json(json_file_path)
categories.printSchema()
categories.show(2)


root
 |-- etag: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- etag: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- kind: string (nullable = true)
 |    |    |-- snippet: struct (nullable = true)
 |    |    |    |-- assignable: boolean (nullable = true)
 |    |    |    |-- channelId: string (nullable = true)
 |    |    |    |-- title: string (nullable = true)
 |-- kind: string (nullable = true)

+--------------------+--------------------+--------------------+
|                etag|               items|                kind|
+--------------------+--------------------+--------------------+
|"ld9biNPKjAjgjV7E...|[{"ld9biNPKjAjgjV...|youtube#videoCate...|
+--------------------+--------------------+--------------------+



In [3]:
popular_channels = df.where(df.channel_title.isNotNull()) \
                    .groupBy("channel_title") \
                    .count() \
                    .orderBy(F.desc("count"))
popular_channels.show(truncate=False)

+----------------------------------+-----+
|channel_title                     |count|
+----------------------------------+-----+
|SET India                         |192  |
|MSNBC                             |189  |
|FBE                               |188  |
|The Young Turks                   |186  |
|REACT                             |183  |
|VikatanTV                         |182  |
|CNN                               |182  |
|The Late Show with Stephen Colbert|172  |
|RadaanMedia                       |168  |
|ARY Digital                       |168  |
|Philip DeFranco                   |167  |
|MLG Highlights                    |165  |
|Comment Awards                    |163  |
|BuzzFeedVideo                     |160  |
|Good Mythical Morning             |157  |
|CollegeHumor                      |153  |
|TheEllenShow                      |150  |
|Breakfast Club Power 105.1 FM     |149  |
|The View                          |149  |
|Late Night with Seth Meyers       |148  |
+----------

In [14]:
view_count = df.groupBy("title") \
            .agg(F.format_number(F.avg(df.likes), 0).alias("avg_likes"),
                F.format_number(F.avg(df.dislikes), 0).alias("avg_dislikes")) \
            .orderBy(F.desc("avg_dislikes"))
view_count.show()

+-----------------------+---------+------------+
|                  title|avg_likes|avg_dislikes|
+-----------------------+---------+------------+
|   What Trump Pullin...|   38,317|         999|
|   Migos x Tasty Whi...|   20,856|         998|
|   Dr. Phil Takes th...|   44,073|         998|
|   Top 5 WWDC 2018 A...|   53,836|         998|
|   The Lion King: Is...|   11,883|         997|
|    Deep Fried Dry Ice?|    8,017|         997|
|     TODAY IS THE DAY!!|   29,007|         997|
|   مسلسل رحيم الحلقة...|    9,469|         997|
|   FT Manchester Utd...|   10,484|         996|
|   Packers vs. Steel...|   11,466|         996|
|   #ScamLifeGuru Tat...|    2,074|         996|
|   You NEED to know ...|    3,840|         996|
|   LA LOVE ARMY EN A...|   95,482|         996|
|   FBI Reveals They ...|   47,290|         996|
|   God of War – Stor...|   77,119|         996|
|   Priyamanaval Epis...|    2,867|         996|
|   Thanksgiving And ...|   16,662|         995|
|[MV] CLC (씨엘씨) _ ..